In [144]:
import pandas as pd
import requests
import json
import csv
import re
import warnings
warnings.filterwarnings('ignore')

### Etapa 1: Entendendo os dados 🎲
Objetivo: nessa etapa, você deve somente ingerir dados da API do randomuser.me e observar o formato dos dados, tentando imaginar como eles poderiam ser usados para construir uma tabela.

Descrição da solução: a solução dessa etapa consiste em uma função para consumir a API na URL https://randomuser.me/api/ e retornar um dicionário com os dados.

Links úteis:

Documentação da API: https://randomuser.me/documentation

Introdução a ingestão de dados via API: https://www.dataquest.io/blog/python-api-tutorial/

In [ ]:
# funcao para ingerir dados via api
def get_data(url):
    response = requests.get(url)
    return response.json()

url = 'https://randomuser.me/api/'

get_data(url)

### Etapa 2: Coletando dados 💾

- **Objetivo:** nessa etapa, você deve coletar dados da API e armazená-los em um arquivo CSV.
- **Descrição da solução:** a solução dessa etapa consiste em uma função para coletar uma quantidade `n` de dados da API (sendo `n` um valor fornecido via parâmetro da função), manipulá-los para montar um `pandas.DataFrame` e salvar o resultado em um arquivo CSV.
- **Links úteis:**
  - Documentação da API: https://randomuser.me/documentation
  - Documentação do Pandas: https://pandas.pydata.org/docs/
- **Dicas:**
  - Para tornar os dados mais fáceis de manipular no futuro, faça com que o `DataFrame` seja "plano", ou seja, cada coluna seja um único atributo do objeto.
  - Para ter dados suficientes para uma análise razoável nas próximas etapas, recomendamos `n>=500`.

In [136]:
# funcao para ingerir dados via api e gerar csv
url = 'https://randomuser.me/api/?results=500'

def request(url):
    response = requests.get(url)
    value = json.loads(response.text) # carrega o json
    # normaliza os dados
    df = pd.json_normalize(value['results'], sep=';')
    df.to_csv('dados/dados_api.csv', index=False, sep=';', encoding='utf-8')

request(url)

### Etapa 3: Manipulando dados 📝

- **Objetivo**: agora, você pode observar que, na base de dados obtida, devido às diferentes nacionalidades dos usuários, os números de telefone e celular têm formatos diferentes. Você deve transformá-los para um formato único, escolhido arbitrariamente.
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e retorna um `pandas.DataFrame` com as mesmas colunas, mas com os números de telefone e celular formatados de forma única.
- **Links úteis:**
- Documentação do Pandas: https://pandas.pydata.org/docs/

In [137]:
df = pd.read_csv('dados/dados_api.csv', sep=';')
df.head()

,gender,email,phone,cell,nat,name;title,name;first,name;last,location;street;number,location;street;name,...,login;sha256,dob;date,dob;age,registered;date,registered;age,id;name,id;value,picture;large,picture;medium,picture;thumbnail
0,male,gustav.nielsen@example.com,92196750,11599051,DK,Mr,Gustav,Nielsen,1420,Blåmejsevej,...,7b46f52172028b69b9a4d945d9d248c897617562eb9611...,1999-01-28T23:37:07.095Z,23,2014-11-22T19:06:19.323Z,7,CPR,280199-0206,https://randomuser.me/api/portraits/men/33.jpg,https://randomuser.me/api/portraits/med/men/33...,https://randomuser.me/api/portraits/thumb/men/...
1,female,osipa.viter@example.com,(098) Z01-6136,(066) C14-8240,UA,Ms,Osipa,Viter,30,Provulok Levitana,...,67d7497fc3408b9ac54f9c0c4915612c5c62704be724b6...,1994-01-10T05:30:09.150Z,28,2006-03-02T04:35:56.885Z,16,NaN,NaN,https://randomuser.me/api/portraits/women/12.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...
2,male,balthasar.steinmetz@example.com,0529-4292407,0178-8739010,DE,Mr,Balthasar,Steinmetz,2323,Brunnenstraße,...,728e5d5547c7260644ae264c73af40133e1f6a46eb78eb...,1951-09-20T13:09:21.909Z,70,2009-03-18T08:12:37.840Z,13,SVNR,79 200951 S 132,https://randomuser.me/api/portraits/men/14.jpg,https://randomuser.me/api/portraits/med/men/14...,https://randomuser.me/api/portraits/thumb/men/...
3,female,ftmhzhr.ysmy@example.com,089-41907347,0992-819-7421,IR,Mrs,فاطمه زهرا,یاسمی,2637,واعظی,...,fed172ff87b1fe46bd45a6734198272487401fe5015038...,1973-04-09T15:39:13.722Z,49,2015-11-03T22:59:57.571Z,6,NaN,NaN,https://randomuser.me/api/portraits/women/8.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...
4,male,rdyn.zraay@example.com,030-96235282,0975-364-9838,IR,Mr,رادین,زارعی,6888,ایران,...,cbe15be67a2d2ac7c6eb16b66e15395060f0de9befe31e...,1970-07-27T16:51:41.927Z,52,2018-06-26T04:03:41.877Z,4,NaN,NaN,https://randomuser.me/api/portraits/men/34.jpg,https://randomuser.me/api/portraits/med/men/34...,https://randomuser.me/api/portraits/thumb/men/...


In [138]:
df.shape

(500, 34)

In [139]:
def format_phone_number(df, column_name):
    if column_name in df.columns:
        def get_number(number):
            """ remover caracteres não numericos"""
            return re.sub(r'[^0-9]', '', number)
        df[column_name] = df[column_name].apply(get_number)

        def format_number(number):
            """ formata o numero de telefone"""
            return f'{number[:2]} {number[2:]}'
        df[column_name] = df[column_name].apply(format_number)
        return df.head()
    else:
        print(f"Coluna {column_name} não encontrada em {df}.")
        return df.head()

        # df[column_name] = df[column_name].apply(lambda x: re.sub(r"[^0-9]", "", x))
        # df[column_name] = df[column_name].str.replace('-', '')
        # """ formatar coluna como xx xxxxxxxxx"""
        

In [140]:
format_phone_number(df, 'phone')
format_phone_number(df, 'cell')

,gender,email,phone,cell,nat,name;title,name;first,name;last,location;street;number,location;street;name,...,login;sha256,dob;date,dob;age,registered;date,registered;age,id;name,id;value,picture;large,picture;medium,picture;thumbnail
0,male,gustav.nielsen@example.com,92 196750,11 599051,DK,Mr,Gustav,Nielsen,1420,Blåmejsevej,...,7b46f52172028b69b9a4d945d9d248c897617562eb9611...,1999-01-28T23:37:07.095Z,23,2014-11-22T19:06:19.323Z,7,CPR,280199-0206,https://randomuser.me/api/portraits/men/33.jpg,https://randomuser.me/api/portraits/med/men/33...,https://randomuser.me/api/portraits/thumb/men/...
1,female,osipa.viter@example.com,09 8016136,06 6148240,UA,Ms,Osipa,Viter,30,Provulok Levitana,...,67d7497fc3408b9ac54f9c0c4915612c5c62704be724b6...,1994-01-10T05:30:09.150Z,28,2006-03-02T04:35:56.885Z,16,NaN,NaN,https://randomuser.me/api/portraits/women/12.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...
2,male,balthasar.steinmetz@example.com,05 294292407,01 788739010,DE,Mr,Balthasar,Steinmetz,2323,Brunnenstraße,...,728e5d5547c7260644ae264c73af40133e1f6a46eb78eb...,1951-09-20T13:09:21.909Z,70,2009-03-18T08:12:37.840Z,13,SVNR,79 200951 S 132,https://randomuser.me/api/portraits/men/14.jpg,https://randomuser.me/api/portraits/med/men/14...,https://randomuser.me/api/portraits/thumb/men/...
3,female,ftmhzhr.ysmy@example.com,08 941907347,09 928197421,IR,Mrs,فاطمه زهرا,یاسمی,2637,واعظی,...,fed172ff87b1fe46bd45a6734198272487401fe5015038...,1973-04-09T15:39:13.722Z,49,2015-11-03T22:59:57.571Z,6,NaN,NaN,https://randomuser.me/api/portraits/women/8.jpg,https://randomuser.me/api/portraits/med/women/...,https://randomuser.me/api/portraits/thumb/wome...
4,male,rdyn.zraay@example.com,03 096235282,09 753649838,IR,Mr,رادین,زارعی,6888,ایران,...,cbe15be67a2d2ac7c6eb16b66e15395060f0de9befe31e...,1970-07-27T16:51:41.927Z,52,2018-06-26T04:03:41.877Z,4,NaN,NaN,https://randomuser.me/api/portraits/men/34.jpg,https://randomuser.me/api/portraits/med/men/34...,https://randomuser.me/api/portraits/thumb/men/...


### Etapa 4: Analisando dados sem agrupamento 📊

- **Objetivo**: com seus dados devidamente tratados, você deve gerar os seguintes itens:
  - Um relatório em texto (não precisa de formatação) contendo:
    - A porcentagem dos usuários por gênero
    - A porcentagem dos usuários por país
  - Uma imagem contendo um gráfico de distribuição da idade dos usuários (a biblioteca utilizada para o `plot` pode ser qualquer uma).
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e gera dois arquivos: um relatório em texto e outro contendo um gráfico de distribuição da idade dos usuários.
- **Links úteis:**
  - Documentação do Pandas: https://pandas.pydata.org/docs/
  - Documentação do Matplotlib: https://matplotlib.org/
  - Documentação do Seaborn: https://seaborn.pydata.org/

In [141]:
df.columns

Index(['gender', 'email', 'phone', 'cell', 'nat', 'name;title', 'name;first',
       'name;last', 'location;street;number', 'location;street;name',
       'location;city', 'location;state', 'location;country',
       'location;postcode', 'location;coordinates;latitude',
       'location;coordinates;longitude', 'location;timezone;offset',
       'location;timezone;description', 'login;uuid', 'login;username',
       'login;password', 'login;salt', 'login;md5', 'login;sha1',
       'login;sha256', 'dob;date', 'dob;age', 'registered;date',
       'registered;age', 'id;name', 'id;value', 'picture;large',
       'picture;medium', 'picture;thumbnail'],
      dtype='object')

In [142]:
# total de usuarios por genero
df_gender = df.gender.value_counts().rename_axis('Gênero').to_frame('Quantidade').reset_index()

# porcentagem de generos
df_gender["%"] = round(df_gender.Quantidade / df_gender.Quantidade.sum() * 100, 0 ) 

df_gender

,Gênero,Quantidade,%
0,male,258,52.0
1,female,242,48.0


In [145]:
# Porcentagem de usuários por país
df_country = df.nat.value_counts().rename_axis('País').to_frame('Quantidade').reset_index()
df_country["%"] = round( df_country.Quantidade / df_country.Quantidade.sum() * 100, 0 )
# add row com total de usuarios
df_country = df_country.append({'País': 'Total', 'Quantidade': df_country.Quantidade.sum(), '%': df_country['%'].sum()}, ignore_index=True)

df_country

,País,Quantidade,%
0,US,36,7.0
1,FI,32,6.0
2,DK,30,6.0
3,TR,28,6.0
4,FR,27,5.0
5,IE,25,5.0
6,IR,25,5.0
7,RS,25,5.0
8,ES,24,5.0
9,AU,24,5.0


### Etapa 5: Analisando dados com agrupamento 📊

- **Objetivo**: utilizar técnicas de agrupamento para descobrir usuários que moram no mesmo país e estado.
- **Descrição da solução**: uma função que recebe, como parâmetro, um `pandas.DataFrame` e retorna um `pandas.DataFrame` com as mesmas colunas, mas com os dados agrupados por país e estado.
- **Links úteis:**
  - Documentação do Pandas: https://pandas.pydata.org/docs/

In [148]:
# descobrir quais usuarios moram no mesmo país e estado


### Etapa 6 (opcional): Particionando dados 🎼
Objetivo: realizar o particionamento dos dados em formato Hive utilizando as informações de país e estado de cada usuário.

Descrição da solução: uma função que recebe, como parâmetro, um pandas.DataFrame e salva todos os dados em arquivos CSV particionados por país e estado.

Links úteis:

Documentação do Pandas: https://pandas.pydata.org/docs/

Documentação do Hive: https://hive.apache.org/

Documentação do BigQuery para dados particionados em Hive: https://cloud.google.com/bigquery/docs/hive-partitioned-queries-gcs

### Etapa 7: Parametrizando seu código ⚙️
Objetivo: nessa etapa, você deve parametrizar seu código para que ele seja executado com valores diversos fornecidos pelo usuário.

Descrição da solução: a solução dessa etapa consiste em uma função principal que recebe diversos parâmetros e executa as diversas etapas descritas anteriormente em função dos parâmetros fornecidos. Note que essa etapa é crucial para que seu código se torne reutilizável.

Dicas:

Tente pensar no maior número de parâmetros que sejam relevantes para sua pipeline de dados, sem afetar sua funcionalidade.

Colocar valores padrão para alguns desses parâmetros reduz o ônus do usuário de preenchê-los por conta própria.